In [89]:
import re
import numpy as np
import heapq
from functools import reduce
from operator import mul
from bisect import insort
from collections import deque
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0
"""

### Part 1
- Thought immediately went to dijkstra and went ahead with it, only later realized it was unnecessary. BFS will do & be cleaner! I keep overcomplicating things!
- Did it in good time though.

In [123]:
class RAMRun:
    def __init__ (self,txt,side):
        self.obs = [tuple(map(int,line.split(","))) for line in txt.splitlines()]
        self.dirs = [(0,1),(1,0),(-1,0),(0,-1)]
        self.start = (0,0)
        self.end = (side-1,side-1)
        # print(f"start={self.start}, end={self.end}")
        self.positions = set()
        self.side = side
        for i in range(side):
            for j in range(side):
                self.positions.add((i,j))

    def find_path(self, obstacles):
        open = deque([(self.start,0)])
        closed = {self.start}
        # open.append((self.start,0))
        while open:
            current_node = open.popleft() # open is a node=(coords,cost)
            for neighbor_coords in self.neighbors(current_node[0],obstacles,closed):
                if neighbor_coords == self.end:
                    return current_node[1]+1
                else:
                    closed.add(neighbor_coords) # add coords to closed
                    open.append((neighbor_coords,current_node[1]+1)) # add {coords,cost}


    # def dijkstra(self, obstacles):
    #     distances = {node: float('inf') for node in res.positions}
    #     distances[self.start] = 0
    #     queue = [(0, self.start)]
    #     while queue:
    #         current_distance, current_node = heapq.heappop(queue)
    #         if current_distance > distances[current_node]:
    #             continue
    #         for neighbor in self.neighbors(current_node,obstacles):
    #             # print(neighbor)
    #             distance = current_distance + 1
    #             if distance < distances[neighbor]:
    #                 distances[neighbor] = distance
    #                 heapq.heappush(queue, (distance, neighbor))
    #     return distances # returns a map of all nodes on the map and its shortest distance to start
    
    def neighbors(self,node,obs_set,closed):
        nbs = []
        for dir in self.dirs:
            resX,resY = (dir[0]+node[0]),(dir[1]+node[1])
            new_pos = (resX,resY)
            if 0<=resX<self.side and  0<=resY<self.side and new_pos not in obs_set and new_pos not in closed:
                nbs.append(new_pos) # append new pos (x,y)
        # print(nbs)
        return nbs
    
    def find_first_block(self,low,high):
        if high >= low:
            i = (high-low)//2 + low
            shortest_path = self.find_path(self.obs[:i+1])
            if shortest_path is None and self.find_path(self.obs[:i]) is not None:
                return self.obs[i]  
            elif shortest_path is None:
                return self.find_first_block(low,i-1)
            else:
                return self.find_first_block(i+1,high)
        else:
            return None

        # for i in range(len(self.obs)):
        #     print(f"i={i}, obs={self.obs[i]}")
        #     shortest_path = res.find_path(res.obs[:i+1])
        #     if shortest_path is None: 
        #         print(f"path={shortest_path}, node {self.obs[i]} has blocked the path")
        #         return self.obs[i]

# print(len(input.splitlines()))

# tests
test_res = RAMRun(test_input,7)
assert test_res.find_path(test_res.obs[:12]) == 22
assert test_res.find_first_block(0,len(test_res.obs)) == (6,1)

# results
res = RAMRun(input,71)
print("part 1:", res.find_path(res.obs[:1024]))
print("part 2:", res.find_first_block(0,len(res.obs)))

part 1: 306
part 2: (38, 63)


### Part 2 + Takeaways
I bashed the code out in an hour after pulling an all nighter and only now got to clean up and think about complexity. I was lucky that today's puzzle was a simple one! Bless the puzzle gods. In terms of takeaways:
- Again, don't overcomplicating things with dijkstra or even a* when BFS would do. consider the puzzle case carefull before choosing a solution
- Optimization: When it's clear that the answer lies within a for loop lasting at least 4 minutes, spend that 4 minutes to program a binary search instead.